In [29]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
dfTraining = pd.read_csv('labels_training_set.csv')
dfEventos = pd.read_csv('events_up_to_01062018.csv')

/home/federico/.local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (3,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# 1.1 Uso árbol de selección para los modelos

In [3]:
arbolClasificador = tree.DecisionTreeClassifier(random_state=10, \
                                                min_samples_split=10, \
                                                min_weight_fraction_leaf =0.05)
arbolClasificador

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=10,
            min_weight_fraction_leaf=0.05, presort=False, random_state=10,
            splitter='best')

In [4]:
dfPersonModel = dfEventos[['person', 'model']]
dfPersonModel = dfPersonModel.dropna()
dfPersonModel = pd.get_dummies(dfPersonModel, columns=['model']).groupby('person').sum()
dfPersonModel.head()

,model_Asus Live,model_Asus Zenfone 2,model_Asus Zenfone 2 Deluxe,model_Asus Zenfone 2 Laser,"model_Asus Zenfone 2 Laser 6""",model_Asus Zenfone 3 Max 32 GB,model_Asus Zenfone 3 Max 16 GB,model_Asus Zenfone 5,model_Asus Zenfone 6,model_Asus Zenfone Go,...,model_iPhone 6,model_iPhone 6 Plus,model_iPhone 6S,model_iPhone 6S Plus,model_iPhone 7,model_iPhone 7 Plus,model_iPhone 8,model_iPhone 8 Plus,model_iPhone SE,model_iPhone X
person,,,,,,,,,,,,,,,,,,,,,
0008ed71,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
00091926,0,0,0,0,0,0,0,0,0,0,...,5,41,96,51,45,9,0,2,3,1
00091a7a,0,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,1,0
000ba417,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
000c79fe,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,4,0,0,0,0,0


In [5]:
dfTrainingPersonModel = dfTraining.sort_values('person').merge(dfPersonModel, on='person', how='left')
dfTrainingPersonModel.head(10)

,person,label,model_Asus Live,model_Asus Zenfone 2,model_Asus Zenfone 2 Deluxe,model_Asus Zenfone 2 Laser,"model_Asus Zenfone 2 Laser 6""",model_Asus Zenfone 3 Max 32 GB,model_Asus Zenfone 3 Max 16 GB,model_Asus Zenfone 5,...,model_iPhone 6,model_iPhone 6 Plus,model_iPhone 6S,model_iPhone 6S Plus,model_iPhone 7,model_iPhone 7 Plus,model_iPhone 8,model_iPhone 8 Plus,model_iPhone SE,model_iPhone X
0,0008ed71,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,000c79fe,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0
2,001802e4,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
3,0019e639,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,001b0bf9,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,001ca5ee,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,20.0,3.0,3.0,4.0,16.0,7.0,0.0,0.0,0.0,0.0
6,001dfc31,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,6.0,0.0,5.0,0.0,0.0,0.0,6.0,0.0
7,001e9aea,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0020152e,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,002b0188,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
ultimaVisitaPorPersona = dfEventos.groupby('person')['timestamp'].max().to_frame()
dfTrainingPersonModelUltimaVisita = dfTrainingPersonModel.merge(ultimaVisitaPorPersona, on='person')
dfTrainingPersonModelUltimaVisita['month'] = dfTrainingPersonModelUltimaVisita['timestamp'].str[5:7]
dfTrainingPersonModelUltimaVisita['day'] = dfTrainingPersonModelUltimaVisita['timestamp'].str[8:10]
dfTrainingPersonModelUltimaVisita[['day','month']] = dfTrainingPersonModelUltimaVisita[['day','month']].apply(pd.to_numeric)
dfTrainingPersonModelUltimaVisita['last_visit'] = dfTrainingPersonModelUltimaVisita['month']*30 + dfTrainingPersonModelUltimaVisita['day']
dfTrainingPersonModelUltimaVisita = dfTrainingPersonModelUltimaVisita.drop(['timestamp','month','day'], axis=1)

In [14]:
#Agrego columna con ultima compra
ultimaCompraPorPersona = dfEventos.loc[dfEventos['event'] == 'conversion'].\
                            groupby('person')['timestamp'].max().to_frame()
dfTPMuVC = dfTrainingPersonModelUltimaVisita.merge(ultimaCompraPorPersona, how='left', on='person')
dfTPMuVC = dfTPMuVC.fillna('000000000000000000')
dfTPMuVC['month'] = dfTPMuVC['timestamp'].str[5:7]
dfTPMuVC['day'] = dfTPMuVC['timestamp'].str[8:10]
dfTPMuVC[['day','month']] = dfTPMuVC[['day','month']].apply(pd.to_numeric)
dfTPMuVC['last_conversion'] = dfTPMuVC['month']*30 + dfTPMuVC['day']
dfTPMuVC = dfTPMuVC.drop(['timestamp','month','day'], axis=1)

In [15]:
df_yMuVC = dfTPMuVC['label']
df_XMuVC = dfTPMuVC.drop(['person', 'label'], axis=1)

X_trainMuVC, X_testMuVC, y_trainMuVC, y_testMuVC = train_test_split(df_XMuVC, df_yMuVC, test_size = 0.5)

In [16]:
arbolClasificador.fit(X_trainMuVC, y_trainMuVC)
entrenamiento = cross_val_score(arbolClasificador, df_XMuVC, df_yMuVC)
entrenamiento.mean()

0.9495209737949358

In [17]:
prediccion = arbolClasificador.predict_proba(X_testMuVC)[:,1]
prediccion.mean()

0.049031981462291284

In [18]:
roc_auc_score(y_testMuVC, prediccion)

0.6736431839015425

# 1.2 Ahora uso GradientBoosting

In [22]:
GBC = GradientBoostingClassifier(random_state=0,learning_rate=0.03,subsample=0.75, min_samples_split=9)
scores = cross_val_score(GBC , df_XMuVC, df_yMuVC, scoring="roc_auc", cv=10)
scores.mean()

0.7106879296368351

In [24]:
GBC.fit(X_trainMuVC, y_trainMuVC)
prediccionGradient = GBC.predict_proba(X_testMuVC)[:,1]
prediccionGradient.mean()

0.04954207014157443

In [25]:
roc_auc_score(y_testMuVC, prediccionGradient)

0.696228788098089

# 1.3 Pruebo con RandomForestClassifier

In [56]:
bosqueClasificador = RandomForestClassifier(n_estimators=10, criterion = 'entropy')

In [57]:
bosqueClasificador.fit(X_trainMuVC, y_trainMuVC)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [58]:
entrenamientoBosque = cross_val_score(bosqueClasificador, df_XMuVC, df_yMuVC)
entrenamientoBosque

array([0.94499382, 0.94669345, 0.94435858])

In [59]:
prediccionBosque = arbolClasificador.predict_proba(X_testMuVC)[:,1]
prediccionBosque

array([0.01550388, 0.01799486, 0.05063291, ..., 0.03497615, 0.06989247,
       0.08297872])

In [60]:
roc_auc_score(y_testMuVC, prediccionGradient)

0.696228788098089

# 2 Vuelvo a aplicar GradientBoosting pero filtrando los modelos que hicieron conversion

In [61]:
personaPorModelo = dfEventos.loc[dfEventos['event'] == 'conversion',['person', 'model']]
personaPorModelo = personaPorModelo.dropna()
personaPorModelo = pd.get_dummies(personaPorModelo, columns=['model']).groupby('person').sum()
personaPorModelo.head()

,model_Asus Zenfone 2,model_Asus Zenfone 2 Laser,"model_Asus Zenfone 2 Laser 6""",model_Asus Zenfone 3 Max 32 GB,model_Asus Zenfone 3 Max 16 GB,model_Asus Zenfone 5,model_LG X Screen,model_LG G3 Beat D724,model_LG G3 D855,model_LG G3 Stylus D690,...,model_iPhone 6,model_iPhone 6 Plus,model_iPhone 6S,model_iPhone 6S Plus,model_iPhone 7,model_iPhone 7 Plus,model_iPhone 8,model_iPhone 8 Plus,model_iPhone SE,model_iPhone X
person,,,,,,,,,,,,,,,,,,,,,
000ba417,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
001001be,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
001804a2,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
0019e639,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
001b0bf9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [62]:
trainingPersonaPorModelo = dfTraining.sort_values('person').merge(personaPorModelo, on='person', how='left')
trainingPersonaPorModelo.head(10)

,person,label,model_Asus Zenfone 2,model_Asus Zenfone 2 Laser,"model_Asus Zenfone 2 Laser 6""",model_Asus Zenfone 3 Max 32 GB,model_Asus Zenfone 3 Max 16 GB,model_Asus Zenfone 5,model_LG X Screen,model_LG G3 Beat D724,...,model_iPhone 6,model_iPhone 6 Plus,model_iPhone 6S,model_iPhone 6S Plus,model_iPhone 7,model_iPhone 7 Plus,model_iPhone 8,model_iPhone 8 Plus,model_iPhone SE,model_iPhone X
0,0008ed71,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,000c79fe,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,001802e4,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0019e639,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,001b0bf9,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,001ca5ee,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,001dfc31,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,001e9aea,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,0020152e,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,002b0188,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [91]:
df_ytPM = trainingPersonaPorModelo['label']
df_XtPM = trainingPersonaPorModelo.drop(['person', 'label'], axis=1)

X_trainTPM, X_testTPM, y_trainTPM, y_testTPM = train_test_split(df_XMuVC, df_yMuVC, test_size = 0.3)

In [92]:
GBC.fit(X_trainTPM, y_trainTPM)
prediccionGradient = GBC.predict_proba(X_testTPM)[:,1]
prediccionGradient.mean()

0.050540297108636195

In [95]:
roc_auc_score(y_testTPM, prediccionGradient)

0.7207387084556991

Combino la feature anterior con la de últimas visitas y las últimas compras

In [73]:
#ultimaVisitaPorPersona = dfEventos.groupby('person')['timestamp'].max().to_frame()
trainingPersonaPorModeloUltimaVisita = trainingPersonaPorModelo.merge(ultimaVisitaPorPersona, on='person')
trainingPersonaPorModeloUltimaVisita['month'] = trainingPersonaPorModeloUltimaVisita['timestamp'].str[5:7]
trainingPersonaPorModeloUltimaVisita['day'] = trainingPersonaPorModeloUltimaVisita['timestamp'].str[8:10]
trainingPersonaPorModeloUltimaVisita[['day','month']] = trainingPersonaPorModeloUltimaVisita[['day','month']].apply(pd.to_numeric)
trainingPersonaPorModeloUltimaVisita['last_visit'] = trainingPersonaPorModeloUltimaVisita['month']*30 + trainingPersonaPorModeloUltimaVisita['day']
trainingPersonaPorModeloUltimaVisita = trainingPersonaPorModeloUltimaVisita.drop(['timestamp','month','day'], axis=1)

In [74]:
#Agrego columna con ultima compra
#ultimaCompraPorPersona = dfEventos.loc[dfEventos['event'] == 'conversion'].\
#                            groupby('person')['timestamp'].max().to_frame()
tPMuVC = trainingPersonaPorModeloUltimaVisita.merge(ultimaCompraPorPersona, how='left', on='person')
tPMuVC = tPMuVC.fillna('000000000000000000')
tPMuVC['month'] = tPMuVC['timestamp'].str[5:7]
tPMuVC['day'] = tPMuVC['timestamp'].str[8:10]
tPMuVC[['day','month']] = tPMuVC[['day','month']].apply(pd.to_numeric)
tPMuVC['last_conversion'] = tPMuVC['month']*30 + tPMuVC['day']
tPMuVC = tPMuVC.drop(['timestamp','month','day'], axis=1)

In [102]:
df_ytPMuVC = tPMuVC['label']
df_XtPMuVC = tPMuVC.drop(['person', 'label'], axis=1)

X_traintPMuVC, X_testtPMuVC, y_traintPMuVC, y_testtPMuVC = train_test_split(df_XtPMuVC, df_ytPMuVC, test_size = 0.2)

In [103]:
GBC.fit(X_traintPMuVC, y_traintPMuVC)
prediccionGradientb = GBC.predict_proba(X_testtPMuVC)[:,1]
prediccionGradientb.mean()

0.05025272106518484

In [104]:
roc_auc_score(y_testtPMuVC, prediccionGradientb)

0.7235765726335893